<h1><center>Laboratorio 5: La desperación de Mr. Cheems 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Catherine Benavides
- Ayudante: Nicolás Ojeda, Eduardo Moya

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1:Manuel Zamorano
- Nombre de alumno 2:Javier Urrutia


### **Link de repositorio de GitHub:** `https://github.com/JUrrutia10/Laboratorios-MDS`

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar procesos más limpios en Feature Engineering.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)

import plotly.express as px
import plotly.graph_objects as go

import sklearn

# Librerias utiles
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [9]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Feature engineering en datos de retail 🛍️

### 0. Cargar Dataset

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

Mr. Cheems, gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas. En una reunión, Mr Cheems le comenta que la calidad de sus datos no es muy buena, por lo que le solicita a usted que limpie su base de datos y cree nuevos atributos relevantes para el negocio.

Por ello, el área de ventas les entrega archivo llamado `online_retail_data.pickle` el cual usted decide cargar a continuación.

In [21]:
path = 'C:\\Users\\jeus8\\Laboratorios-MDS\\lab5\\online_retail_data.pickle'
df_retail = pd.read_pickle(path)
display(df_retail.head())
df_retail.dtypes

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,ID489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12.0,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,ID489434,79323P,PINK CHERRY LIGHTS,12.0,NaT,6.75,13085.0,United Kingdom
2,ID489434,79323W,WHITE CHERRY LIGHTS,12.0,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,ID489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48.0,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,ID489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24.0,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


Invoice                object
StockCode            category
Description            object
Quantity              float64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID          category
Country              category
dtype: object

### 1. Función para explorar características [5 puntos]

<p align="center">
  <img width=300 src="https://editor.analyticsvidhya.com/uploads/47389meme.png">
</p>




Tras inspeccionar brevemente los datos proporcionados, usted decide crear una función que realice lo siguiente:
- Plotee un histograma para las variables precios y cantidad. [3 puntos]
- Imprima un conteo de datos nulos por variable [2 puntos]

NOTA: Para generar los gráficos es **OBLIGATORIO** el uso de plotly.

**Respuesta:**

In [22]:
def explore_data(dataframe_in):

  cols = [
    'Quantity',
    'Price']
  fig1 = px.histogram(dataframe_in[cols].melt(),
               x='value',
               color='variable',
               barmode='group')

  fig1.show()

  # Print de valores nulos
  print('Valores nulos por variable:')
  print(dataframe_in.isnull().sum().sort_values(ascending = False))

In [ ]:
explore_data(df_retail)

### 2. Eliminando outliers [10 puntos]

<p align="center">
  <img width=300 src="https://media.licdn.com/dms/image/C5612AQGdXKCka7HumA/article-cover_image-shrink_600_2000/0/1520056407281?e=2147483647&v=beta&t=VZcfjjzjK4LxXdZkSu1KisWC0Ry8bk4tPCn3R8aYdNM">
</p>




#### 2.1 Creando la clase IQR [5 puntos]

Entre las falencias de los datos, Mr. Cheems le comenta que a veces los operadores no ingresan el precio correcto de los productos. Mr. Cheems le comenta que se dio cuenta de este fenómeno porque hay productos con precios exagerádamente altos o bajos. Por lo cual usted decide eliminar outliers del dataframe a traves del rango intercuartil el cual cuenta con los siguientes pasos:

1. Calcular el primer cuartil $Q1$ y el tercer cuartil $Q3$. Hint: utilice el método `quantile()`

2. Calcular el rango intercuartil (RIC): $RIC = Q3 - Q1$

3. Calcular los límites para identificar outliers:
 - Límite inferior: $~~Q1 - \lambda \cdot RIC$
 - Límite superior: $~~Q3 + \lambda \cdot RIC$

4. Eliminar outliers: Los outliers son los datos que están por debajo del límite inferior o por encima del límite superior.


Para realizar dicha tarea, usted decide crear una clase llamada `IQR()` utilizando `BaseEstimator` y `TransformerMixin` para realizar una transformación de cada una de las columnas numéricas del DataFrame utilizando `ColumnTransformer()` más tarde. Considere que lambda debe ser $\lambda$ un parámetro a definir por el usuario.

Hint: tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer).

**Respuesta:**

In [24]:
class IQR(BaseEstimator, TransformerMixin):
  def __init__(self, lambda_=1.5): # cuanto mayor sea lambda_, menos valores serán considerados como outliers
        self.lambda_ = lambda_
  def fit(self, X, y=None):
    self.Q1 = X.quantile(0.25)
    self.Q3 = X.quantile(0.75)
    self.RIC = self.Q3 - self.Q1
    self.lower_limit = self.Q1 - self.lambda_ * self.RIC
    self.upper_limit = self.Q3 + self.lambda_ * self.RIC
    return self
  def transform(self, X): # filtra los datos para eliminar los outliers en cada columna
    X_transformed = X.copy()
    for col in X.columns:
      X_transformed[col] = X_transformed[col][(X_transformed[col] >= self.lower_limit[col]) &
                                                    (X_transformed[col] <= self.upper_limit[col])]
    return X_transformed

#### 2.2 Creación del Pipeline [5 puntos]

Para comenzar introduciéndose en el uso de pipeline, usted decide definir un pipeline con el Transformer previamente definido. Además, usted decide visualizar cómo cambia la distribución de las variables Precio y Cantidad antes y despues de aplicar IQR. Para ello, usted aplica los siguientes pasos:

- Definir un pipeline llamado `numeric_transformations` para las variables precio y cantidad con la transformación IQR. [1 punto]
- Defina un column transformer que aplique `numeric_transformations` para las variables numéricas y `passthrough` para las variables categóricas. Ver hint al final. [1 puntos]
- Defina el dataframe `df_iqr` aplicado el column transformer a los datos proporcionados por Mr. Cheems considerando un valor de $\lambda$ que tenga un desempeño aceptable para ambas variables. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_iqr`.  [1 punto]
- Reportar los cambios observados en la distribución de las variables. ¿Qué sucede al aumentar el valor de lambda? [1 punto]


HINT: El transformador 'passthrough' está predefinido y es una opción que puedes usar para las columnas que no deseas transformar. Al especificar 'passthrough' para una parte de tu ColumnTransformer, las columnas correspondientes pasarán a través del ColumnTransformer sin ninguna modificación.

**Respuesta:**

Apóyese de la siguiente estructura para su respuesta:

In [ ]:
# Definicion las variables que pasarán por cada pipeline
numerical_columns = ['Quantity', 'Price']
categorical_columns = ['Invoice', 'StockCode', 'Description', 'InvoiceDate', 'Customer ID', 'Country']


# Definicion del pipeline
numeric_transformations = Pipeline([('iqr', IQR(lambda_=1.0))])

# Configuracio el ColumnTransformer
column_transformer = ColumnTransformer(transformers=[
    ('numerical', numeric_transformations, numerical_columns),
    ('categorical', 'passthrough', categorical_columns)], remainder='passthrough')

# Aplicamos ColumnTransformer a los datos
df_iqr = column_transformer.fit_transform(df_retail).copy()
df_iqr = pd.DataFrame(df_iqr).copy() # Al parecer no devulve un dataframe
df_iqr.columns = numerical_columns + categorical_columns # Por alguna razón no devuelve los nombres de columnas

# Gráficos
print('Exploración data df_retail')
explore_data(df_retail)
print('Exploración data df_iqr')
explore_data(df_iqr)

Lo primero que se puede observar es que al elimnar los outliers, aumenta la cantidad de nulos en las variables Price Y Quantity. Esto se da ya que los valores que se consideran outliers son reeplazados por NaNs.

Por otro lado, al aumentar lambda, se puede ver que menos valores se consideran outliers. Es decir, cuanto menor sea lambda, más valores serán considerados como outliers.

### 3. Agregando un imputer al pipeline [10 puntos]



<p align="center">
  <img width=300 src="https://media.makeameme.org/created/hmm-there-is.jpg">
</p>

Para continuar con la limpieza del dataframe usted decide imputar los datos nulos de las variables numéricas, para lo cual decide realizar las siguientes tareas:

1. Crear un pipeline para variables categóricas llamado `categoric_transformations` con un paso llamado `mode_imputer`, en el cual se imputen los datos faltantes por la moda [link](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [1 punto]
2. Agregar al pipeline `numeric_transformations` un paso llamado `mean_imputer`, en el cual se imputen los datos por la media [link](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [1 punto]
3. Aplicar el ColumnTransformer actualizado a `df_retail`, creando un dataframe llamado `df_mean_imputer`. [1 punto]
4. Comparar los resultados de `explore_data` en `df_mean_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
5. Cambiar el imputer de `numeric_transformations` por `knn_imputer` [link](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) y definir un nuevo dataframe llamado `df_knn_imputer`, aplicando el nuevo ColumnTransformer a `df_retail`. [1 punto]
6. Comprar los resultados de `explore_data` en `df_knn_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
7. Comprar los resultados de `explore_data` en `df_knn_imputer` y `df_mean_imputer`. ¿Cuál método de imputación es mejor? Deje el método escogido en el ColumnTransformer. [2 puntos]

**Respuesta:**

Para hacer esta parte se hara un sample de la data ya que es muy alta la cantidad de datos para ejecutar el KNN inputer y los tiempos de ejecucion se disparan.

In [26]:
tamano_muestra = int(len(df_retail) * 0.025)# el 2.5% son alrededor de 10.000 datos
df_retail_sample = df_retail.sample(n=tamano_muestra, random_state=2024)

In [27]:
df_retail_sample.describe()
len(df_retail_sample)

10023

Se aplica de nuevo el iqr para que sea comparable

In [28]:
df_iqr = column_transformer.fit_transform(df_retail_sample).copy()
df_iqr = pd.DataFrame(df_iqr).copy() # Al parecer no devulve un dataframe
df_iqr.columns = numerical_columns + categorical_columns # Por alguna razón no devuelve los nombres de columnas

In [29]:
from sklearn import set_config

from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

categoric_transformations = Pipeline([
    ('mode_imputer', SimpleImputer(strategy='most_frequent'))
])

numeric_transformations.steps.append(('mean_imputer', SimpleImputer(strategy='mean')))





column_transformer = ColumnTransformer(transformers=[
    ('numerical', numeric_transformations, numerical_columns),
    ('categorical', categoric_transformations, categorical_columns)
])

df_mean_imputer = column_transformer.fit_transform(df_retail_sample)
df_mean_imputer = pd.DataFrame(df_mean_imputer, columns=numerical_columns + categorical_columns)







In [30]:
set_config(display="diagram")
numeric_transformations

Pipeline(steps=[('iqr', IQR(lambda_=1.0)), ('mean_imputer', SimpleImputer())])

In [31]:
from sklearn.impute import KNNImputer


KNNimputer = KNNImputer(n_neighbors=2, weights="uniform")

numeric_transformations.steps.insert(1,['knn_imputer', KNNimputer])

numeric_transformations.steps.pop(2)


set_config(display="diagram")
numeric_transformations



Pipeline(steps=[('iqr', IQR(lambda_=1.0)),
                ['knn_imputer', KNNImputer(n_neighbors=2)]])

In [32]:
column_transformer = ColumnTransformer(transformers=[
    ('numerical', numeric_transformations, numerical_columns),
    ('categorical', categoric_transformations, categorical_columns)
])
df_knn_imputer = column_transformer.fit_transform(df_retail_sample)
df_knn_imputer = pd.DataFrame(df_knn_imputer, columns=numerical_columns + categorical_columns)



In [ ]:
print('Exploración data df_iqr')
explore_data(df_iqr)
print('Exploración data df_mean_imputer')
explore_data(df_mean_imputer)

print('Exploración data df_knn_imputer')
explore_data(df_knn_imputer)

*Escriba su respuesta aquí*

Se puede notar en los gráficos que comparando el pipeline de knniputer con el de meaninputer, la media de las distribuciones es diferentes, siendo el del Knninputer menor y más similar al que se consigue aplicando el iqr solamente, aunque según las tablas ambos modelos reducen totalmente los valores nulos.


Luego, comparando el KNNinputer con el transformer que solo tiene el iqr, se puede notar que tienen una distribución similar aunque la cantidad de datos es diferente, teniendo el KNNinputer una cantidad mayor esto hace sentido ya que se consiguió disminuir totalmente los datos faltantes.


### 4. Creación de nuevas features [20 puntos]

#### 4.1 Definicion de LRMFP [10 puntos]
(3 puntos por cada custom feature)

Dado que Mr. Lepin está interesado en obtener nuevos atributos relevantes para su negocio, su equipo de expertos sugiere la construcción de variables **LRMFP**, las que se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [34]:
def custom_features(dataframe_in):
    # 1. Calcular la longitud (Length) para cada cliente
    length_df = dataframe_in.groupby('Customer ID')['InvoiceDate'].apply(lambda x: (x.max() - x.min()).days).reset_index()
    length_df.columns = ['Customer ID', 'Length']

    # 2. Calcular la recencia (Recency) para cada cliente
    recency_df = dataframe_in.groupby('Customer ID')['InvoiceDate'].max().reset_index()
    recency_df['Recency'] = (dataframe_in['InvoiceDate'].max() - recency_df['InvoiceDate']).dt.days
    recency_df = recency_df.drop('InvoiceDate', axis=1)

    # 3. Calcular el valor monetario medio (Monetary) para cada cliente
    monetary_df =dataframe_in.groupby('Customer ID')['Price'].mean().reset_index()
    monetary_df.columns = ['Customer ID', 'Monetary']

    # 4. Calcular la frecuencia (Frequency) para cada cliente
    frequency_df =dataframe_in.groupby('Customer ID').size().reset_index(name='Frequency')

    # 5. Calcular la periodicidad (Periodicity) para cada cliente
    dataframe_in['IVT'] = dataframe_in.groupby('Customer ID')['InvoiceDate'].diff().dt.days
    periodicity_df = dataframe_in.groupby('Customer ID')['IVT'].std().reset_index()
    periodicity_df.columns = ['Customer ID', 'Periodicity']

    # Combinar los resultados en un solo DataFrame
    LRMFP_df = length_df.merge(recency_df, on='Customer ID', how='left')
    LRMFP_df = LRMFP_df.merge(monetary_df, on='Customer ID', how='left')
    LRMFP_df = LRMFP_df.merge(frequency_df, on='Customer ID', how='left')
    LRMFP_df = LRMFP_df.merge(periodicity_df, on='Customer ID', how='left')

    return LRMFP_df

In [35]:
LRMFP_df = custom_features(df_retail).copy()

C:\Users\jeus8\AppData\Local\Temp\ipykernel_12132\4114337282.py:3: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\jeus8\AppData\Local\Temp\ipykernel_12132\4114337282.py:7: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\jeus8\AppData\Local\Temp\ipykernel_12132\4114337282.py:12: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\jeus8\AppData\Local\Temp\ipykernel_12132\4114337282.py:16: FutureWarning:



In [36]:
LRMFP_df

,Customer ID,Length,Recency,Monetary,Frequency,Periodicity
0,12346.0,196.0,164.0,6.253333,33,22.401662
1,12347.0,37.0,2.0,2.164493,71,4.554385
2,12348.0,0.0,73.0,0.719500,20,0.000000
3,12349.0,181.0,42.0,8.577400,102,16.363451
4,12351.0,0.0,10.0,2.355238,21,0.000000
...,...,...,...,...,...,...
4309,18283.0,275.0,17.0,2.211887,217,12.006578
4310,18284.0,0.0,66.0,3.279259,28,0.000000
4311,18285.0,0.0,295.0,8.350000,12,0.000000
4312,18286.0,247.0,111.0,4.280806,67,30.875000


#### 4.2 Agregando las custom features [10 puntos]

Ahora, usted decide agregar al pipeline las nuevas variables creadas, para lo cual realiza las siguientes tareas:

1. Cree un nuevo pipeline llamado `retail_pipeline` que encapsule el ColumnTransformer y calcule las LRMFP. El primer paso del pipeline llámelo  `col_tranformer` y el segundo paso llámelo `custom_features`, incorpora las nuevas variables al dataframe. Hint: les puede ser útil investigar [este](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) método. [1 punto]
2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems, creando un nuevo dataframe llamado `df_custom`. [1 punto]
3. Explorar la distribución de las nuevas variables con `explore_data` y comentar brevemente (2-3 líneas) características de cada custom feature. [5 puntos]
5. Entregar un insight para el negocio en base a las nuevas variables. [3 puntos]

**Respuesta**

In [39]:
numeric_transformations = Pipeline([('iqr', IQR(lambda_=1.0))])
numeric_transformations.steps.append(('mean_imputer', SimpleImputer(strategy='mean')))



In [42]:
type(df_retail)

pandas.core.frame.DataFrame

In [46]:
column_transformer = ColumnTransformer(transformers=[
    ('numerical', numeric_transformations, numerical_columns),
    ('categorical', categoric_transformations, categorical_columns)
])

# Creación del pipeline retail_pipeline
retail_pipeline = Pipeline([
    ('col_transformer', column_transformer),
    ('to_dataframe', FunctionTransformer(func=lambda x: pd.DataFrame(x, columns=numerical_columns + categorical_columns))),
    ('custom_features',  FunctionTransformer(func=custom_features))
])

# 2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems
df_custom = retail_pipeline.fit_transform(df_retail)

df_custom = pd.DataFrame(df_custom, columns=LRMFP_df.columns)

### 5. MinMax Scaler [10 puntos]

<p align="center">
  <img width=300 src="https://i.imgflip.com/1fsprn.jpg">
</p>


#### 5.1 Definición del Column Transformer [5 puntos]

Construya una clase llamada `MinMax()` para realizar una transformación de cada una de las columnas de un DataFrame utilizando `ColumnTransformer()`. Recuerde  usar `BaseEstimator` y `TransformerMixin`.


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$


Consulte el siguiente [link](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer) si tiene dudas sobre la creación de custom transformers.

**Respuesta:**

In [47]:
class MinMax(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # No es necesario calcular nada en el ajuste, así que simplemente regresamos el objeto
        return self

    def transform(self, X):
        # Para cada columna, calculamos el mínimo y el máximo
        min_vals = X.min()
        max_vals = X.max()

        # Aplicamos la fórmula de Min-Max scaling a cada columna
        X_transformed = (X - min_vals) / (max_vals - min_vals)

        return X_transformed

In [48]:
df = pd.DataFrame({'A': [1, 2, 3, 4, 5], 'B': [10, 20, 30, 40, 50]})
min_max_transformer = MinMax()
df_transformed = min_max_transformer.fit_transform(df[['A', 'B']])
print(df_transformed)

      A     B
0  0.00  0.00
1  0.25  0.25
2  0.50  0.50
3  0.75  0.75
4  1.00  1.00


#### 5.2 Incorporando MinMax al pipeline [5 puntos]

Ahora, usted decide agregar el escalamiento al pipeline, para lo que decide seguir los siguientes pasos:

- Agregar el paso `minmax` al pipeline `numeric_transformations`, haciendo uso de la clase creada. [1 punto]
- Defina el dataframe `df_minmax` aplicando el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_minmax`. [1 punto]
- Reportar los cambios observados en la distribución de las variables.  [2 puntos]

**Respuesta:**

In [ ]:
# Agregar el paso minmax al pipeline numeric_transformations
numeric_transformations.steps.append(('minmax', MinMax()))

# Aplicar el ColumnTransformer actualizado a los datos
df_minmax = column_transformer.fit_transform(df_retail)
df_minmax = pd.DataFrame(df_minmax, columns=numerical_columns + categorical_columns)

# Usar explore_data en df_retail y en df_minmax
print('Exploración data df_retail')
explore_data(df_retail)
print('Exploración data df_minmax')
explore_data(df_minmax)

Se puede observar que luego de utilizar el pipeline, la distribución de los datos mejoró de manera abrupta, pues loas datos dejaron de concentrarse a la izquierda. Con el nuevo pipeline, los valores (value) parecieran estar normalizados, pues se mueven entre 0 y 1, y no entre 0 y 18k como en el caso de df_retail.

### 6. Pregunta teórica [5 puntos]

Finalmente, explíquele a Mr. Cheems porqué es útil la creación de pipelines al momento de hacer Feature Engineering en Machine Learning.

**Respuesta:**

Aplicar pipeline para hacer feature engineering permite tener un proceso más ordenado y estandarizado para aplicar las transformaciones a los datos con los que se está trabajando, de esta forma es mas facil y rapido poder aplicar las mismas transformaciones a diferentes conjuntos de datos o en diferentes momentos, también tiene la facilidad de poder definir las transformaciones según el tipo de dato o de columnas, además de poder editar el flujo y orden de estas dentro de los pipelines.

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>